In [2]:
import yaml
import os
import pathlib
ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)


In [3]:
DATA_PATH = ROOT / 'data' / 'seminar2'

In [8]:
#3
# Variables from config
API_KEY = config['api']['keys'][2]
API_VERSION = config['api']['api_version']
API_SERVICE_NAME = config['api']['api_service_name']

publishedAfter = config['api']['parameters']['date_after']
publishedBefore = config['api']['parameters']['date_before']



In [9]:
# 4

import googleapiclient.discovery as api

In [10]:
# 5

youtube = api.build(
    API_SERVICE_NAME,
    API_VERSION,
    developerKey = API_KEY
)

In [11]:
# 6
# Одинарный запрос к API
import pandas as pd

# Для поиска
q = "корейская косметика"
search_type = "channel"


request = youtube.search().list(
    part="snippet",
    q=q,
    type=search_type,
    maxResults = 50,
    regionCode="RU"
)

response = request.execute()

response

{'kind': 'youtube#searchListResponse',
 'etag': 'nYkyUc8YlPI2-SS6Ch6IQ57FAnI',
 'nextPageToken': 'CDIQAA',
 'regionCode': 'RU',
 'pageInfo': {'totalResults': 6545, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'roZ6EFYKGbXE-T9_3XN7Xz5TXPI',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCXNXpk6BPZO-M-fYyvfF06Q'},
   'snippet': {'publishedAt': '2016-08-08T18:44:38Z',
    'channelId': 'UCXNXpk6BPZO-M-fYyvfF06Q',
    'title': 'Bo Hyun Kim',
    'description': 'Здесь я делюсь с Вами полезной информацией о корейской косметике и о правильном корейском уходе. Я уверена, что ...',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJVedf_bXWhv_yh8D3f0jy73odHzsELfdHwkF5l6Lg=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJVedf_bXWhv_yh8D3f0jy73odHzsELfdHwkF5l6Lg=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJVedf_bXWhv_yh8D3f0jy73odHzsELfdHwkF5l6Lg=s800-c-k-c0xff

In [12]:
# 7
df_supplemented = pd.json_normalize(response['items'])

In [13]:
# 8
# Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            part ="snippet",
            q = q,
            type = search_type,
            maxResults=50,
            pageToken = response['nextPageToken']
        )
    response = request.execute()

    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])



print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates('id.channelId'))}")

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Итерация БЕЗ включения аргумента order №6
Итерация БЕЗ включения аргумента order №7
Итерация БЕЗ включения аргумента order №8
Итерация БЕЗ включения аргумента order №9
Искомых объектов 6754 а найденных БЕЗ включения каких-либо значений аргумента order 488


In [14]:
# 9
df_supplemented = df_supplemented.drop_duplicates()

df_supplemented = df_supplemented.reset_index(drop=True)

print('Уникальных объектов', len(df_supplemented))

Уникальных объектов 488


In [15]:
# 9

import os


if not os.path.exists(ROOT / DATA_PATH):
    os.makedirs(ROOT / DATA_PATH)

df_supplemented.to_excel(ROOT / DATA_PATH / f'{search_type.capitalize()}_No_sorting.xlsx', index=False)